# Imports

In [1]:
import os
from sys import path
path.append('../')

import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers
from src.dataset import PneumoniaDetectionChallenge

# Dataset loading for training

In [2]:
DATASET_PATH = '../datasets/rsna-pneumonia-detection-challenge/'

In [3]:
pdc = PneumoniaDetectionChallenge(DATASET_PATH)
x_train, y_train = pdc.load_train_data()

In [4]:
x_train = x_train / 255.0

In [5]:
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)

# Mdel definition

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        36928     
Total params: 56,320
Trainable params: 56,320
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [8]:
model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Model training

In [9]:
history = model.fit(x_train, y_train, epochs=15)

Epoch 1/15
626/626 [==============================] - 13s 11ms/step - loss: 2.1883 - accuracy: 0.8604
Epoch 2/15
626/626 [==============================] - 7s 11ms/step - loss: 1.9952 - accuracy: 0.8762
Epoch 3/15
626/626 [==============================] - 7s 11ms/step - loss: 1.9891 - accuracy: 0.8766
Epoch 4/15
626/626 [==============================] - 7s 11ms/step - loss: 1.9614 - accuracy: 0.8783
Epoch 5/15
626/626 [==============================] - 7s 11ms/step - loss: 2.0632 - accuracy: 0.8720
Epoch 6/15
626/626 [==============================] - 7s 11ms/step - loss: 2.0635 - accuracy: 0.8720
Epoch 7/15
626/626 [==============================] - 7s 11ms/step - loss: 2.1116 - accuracy: 0.8690
Epoch 8/15
626/626 [==============================] - 7s 11ms/step - loss: 2.0498 - accuracy: 0.8728
Epoch 9/15
626/626 [==============================] - 7s 11ms/step - loss: 1.9915 - accuracy: 0.8764
Epoch 10/15
626/626 [==============================] - 7s 11ms/step - loss: 2.0672 - accur

In [10]:
del x_train
del y_train

# Evaluating the model

In [11]:
x_test, y_test = pdc.load_test_data()

In [12]:
x_test = x_test / 255.0

In [13]:
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)

In [14]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)

209/209 [==============================] - 1s 5ms/step - loss: 2.1141 - accuracy: 0.8688
